In [28]:
# import packages

In [29]:
from pandasql import sqldf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import io
import requests

In [30]:
#matplotlib inline

In [31]:
%matplotlib inline

In [32]:
#project csv

In [33]:
project_csv = pd.read_csv('../data/Hot-Stuff.csv')
print(project_csv)

                                                      url      WeekID  \
0       http://www.billboard.com/charts/hot-100/1965-0...   7/17/1965   
1       http://www.billboard.com/charts/hot-100/1965-0...   7/24/1965   
2       http://www.billboard.com/charts/hot-100/1965-0...   7/31/1965   
3       http://www.billboard.com/charts/hot-100/1965-0...    8/7/1965   
4       http://www.billboard.com/charts/hot-100/1965-0...   8/14/1965   
...                                                   ...         ...   
327890  https://www.billboard.com/charts/hot-100/2018-...  10/20/2018   
327891  http://www.billboard.com/charts/hot-100/1977-0...   5/21/1977   
327892  http://www.billboard.com/charts/hot-100/1981-0...   5/23/1981   
327893  http://www.billboard.com/charts/hot-100/1973-0...   9/29/1973   
327894  https://www.billboard.com/charts/hot-100/2020-...  11/28/2020   

        Week Position                      Song                  Performer  \
0                  34    Don't Just Stand The

In [34]:
#Auth

In [35]:
!pip install requests

In [36]:
import base64
import datetime
from urllib.parse import urlencode

In [37]:
client_id = '6e24789ec7874259a220f747fcd7e93e'
client_secret = '9bd8ff1ad2f34ef6a87f00124be070b2'

In [38]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"



    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def getClientCreds(self):
        '''Returns b64 encoded string'''
        client_id = self.client_id
        client_secret = self.client_secret

        if client_id == None or client_secret == None:
            raise Exception('Must set a client id and secret')

        client_creds  = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    def getTokenHeader(self):
        client_creds_b64 = self.getClientCreds()
        return {
            'Authorization':f"Basic {client_creds_b64}"
        }

    def getTokenData(self):
        return {
            "grant_type":"client_credentials"
        }

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.getTokenData()
        token_header = self.getTokenHeader()

        r = requests.post(token_url, data=token_data, headers=token_header)

        if r.status_code  not in range(200,299):
            raise Exception('Could not authenticate client.')
            #return False

        now = datetime.datetime.now()
        token_response_data = r.json()
        access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now


        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now= datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}", 
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers = headers)
        if r.status_code not in range (200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id ):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id ):
        return self.get_resource(_id, resource_type='artists')
    
    
    def base_search(self, query_params): #type
        access_token = self.get_access_token()
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_URL = f"{endpoint}?{query_params}"
        r = requests.get(lookup_URL, headers=headers)
        if r.status_code not in range (200, 299):
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if query == None:
            raise Exception('A query is required')
        if isinstance(query, dict):
            query = ' '.join([f'{k}:{v}' for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == 'or' or operator.lower() == 'not':
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f'{query} {operator} {operator_query}'
        query_params = urlencode({'q': query, 'type': search_type.lower()})
        return self.base_search(query_params)

In [39]:
spotify = SpotifyAPI(client_id, client_secret)

In [ ]:
#search

In [46]:
spotify.search({'track': 'Radioactive', 'artist': 'Imagine'}, search_type ='track')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3ARadioactive+artist%3AImagine&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/53XhwfbYqKCa1cC15pYq2q'},
       'href': 'https://api.spotify.com/v1/artists/53XhwfbYqKCa1cC15pYq2q',
       'id': '53XhwfbYqKCa1cC15pYq2q',
       'name': 'Imagine Dragons',
       'type': 'artist',
       'uri': 'spotify:artist:53XhwfbYqKCa1cC15pYq2q'}],
     'available_markets': ['CA', 'US'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/6htgf3qv7vGcsdxLCDxKp8'},
     'href': 'https://api.spotify.com/v1/albums/6htgf3qv7vGcsdxLCDxKp8',
     'id': '6htgf3qv7vGcsdxLCDxKp8',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273b2b2747c89d2157b0b29fb6a',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02b2b2747c89d2157b0b29fb6a',
      

In [ ]:
#remove items

In [45]:
spotify.search(query='Danger', operator='NOT', operator_query='Zone', search_type = 'Track')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Danger+NOT+Zone&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4oUHIQIBe0LHzYfvXNW4QM'},
       'href': 'https://api.spotify.com/v1/artists/4oUHIQIBe0LHzYfvXNW4QM',
       'id': '4oUHIQIBe0LHzYfvXNW4QM',
       'name': 'Morgan Wallen',
       'type': 'artist',
       'uri': 'spotify:artist:4oUHIQIBe0LHzYfvXNW4QM'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE'